In [2]:
import pandas as pd
import numpy as np
import collections

import math

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from ANP import *

import os
import math
from dataclasses import dataclass

import torch
from botorch.acquisition import qExpectedImprovement
from botorch.fit import fit_gpytorch_mll
from botorch.generation import MaxPosteriorSampling
from botorch.models import SingleTaskGP
from botorch.optim import optimize_acqf
from botorch.test_functions import Ackley
from botorch.utils.transforms import unnormalize
from torch.quasirandom import SobolEngine

import gpytorch
from gpytorch.constraints import Interval
from gpytorch.kernels import MaternKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.priors import HorseshoePrior


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float
SMOKE_TEST = os.environ.get("SMOKE_TEST")

C:\Users\82109\AppData\Local\Temp\ipykernel_19548\1035163680.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
fun = Ackley(dim=20, negate=True).to(dtype=dtype, device=device)
fun.bounds[0, :].fill_(-5)
fun.bounds[1, :].fill_(10)
dim = fun.dim
lb, ub = fun.bounds
print(dim)

batch_size = 4
n_init = 2 * dim
max_cholesky_size = float("inf")  # Always use Cholesky

NUM_RESTARTS = 10 if not SMOKE_TEST else 2
RAW_SAMPLES = 512 if not SMOKE_TEST else 4
N_CANDIDATES = min(5000, max(2000, 200 * dim)) if not SMOKE_TEST else 4

20


In [4]:
print(RAW_SAMPLES)

512


In [5]:
def eval_objective(x):
    """This is a helper function we use to unnormalize and evalaute a point"""
    return fun(unnormalize(x, fun.bounds))

def get_initial_points(dim, n_pts, seed=0):
    sobol = SobolEngine(dimension=dim, scramble=True, seed=seed)
    X_init = sobol.draw(n=n_pts).to(dtype=dtype, device=device)
    return X_init

In [6]:
# X_ei = get_initial_points(dim, n_init)
# Y_ei = torch.tensor(
#     [eval_objective(x) for x in X_ei], dtype=dtype, device=device
# ).unsqueeze(-1)
# print(X_ei.size())
# print(Y_ei.size())

### GP-EI

In [7]:
# torch.manual_seed(0)

# X_ei = get_initial_points(dim, n_init)
# Y_ei = torch.tensor(
#     [eval_objective(x) for x in X_ei], dtype=dtype, device=device
# ).unsqueeze(-1)

# while len(Y_ei) < 400:
#     train_Y = (Y_ei - Y_ei.mean()) / Y_ei.std()
#     likelihood = GaussianLikelihood(noise_constraint=Interval(1e-8, 1e-3))
#     model = SingleTaskGP(X_ei, train_Y, likelihood=likelihood)
#     mll = ExactMarginalLogLikelihood(model.likelihood, model)
#     fit_gpytorch_mll(mll)

#     # Create a batch
#     ei = qExpectedImprovement(model, train_Y.max())
#     candidate, acq_value = optimize_acqf(
#         ei,
#         bounds=torch.stack(
#             [
#                 torch.zeros(dim, dtype=dtype, device=device),
#                 torch.ones(dim, dtype=dtype, device=device),
#             ]
#         ),
#         q=batch_size,
#         num_restarts=NUM_RESTARTS,
#         raw_samples=RAW_SAMPLES,
#     )
#     Y_next = torch.tensor(
#         [eval_objective(x) for x in candidate], dtype=dtype, device=device
#     ).unsqueeze(-1)

#     # Append data
#     X_ei = torch.cat((X_ei, candidate), axis=0)
#     Y_ei = torch.cat((Y_ei, Y_next), axis=0)
#     # print(X_ei.size(), Y_ei.size())

#     # Print current status
#     print(f"{len(X_ei)}) Best value: {Y_ei.max().item():.2e}")

### NP-MC

In [8]:
import glob
import numpy as np
import torch
from math import pi
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

class My_Ackley(Dataset):
    def __init__(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y

        # Generate data
        self.data = []
        self.data.append((self.train_x, self.train_y))

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [9]:
# from torch.utils.data import DataLoader
# from neural_process import NeuralProcess
# from training import NeuralProcessTrainer

# ## NP model
# x_dim = 20
# y_dim = 1
# r_dim = 50  # Dimension of representation of context points
# z_dim = 50  # Dimension of sampled latent variable
# h_dim = 50  # Dimension of hidden layers in encoder and decoder
# neuralprocess = NeuralProcess(x_dim, y_dim, r_dim, z_dim, h_dim)

# torch.manual_seed(0)
# X_np = get_initial_points(dim, n_init)
# Y_np = torch.tensor(
#     [eval_objective(x) for x in X_np], dtype=dtype, device=device
# ).unsqueeze(-1)

# batch_size = 2
# num_context = 4
# num_target = 4

# ackley_dataset = My_Ackley(train_x = X_np, train_y = Y_np)
# data_loader = DataLoader(ackley_dataset, batch_size=batch_size, shuffle=True)

# optimizer = torch.optim.Adam(neuralprocess.parameters(), lr=3e-4)
# np_trainer = NeuralProcessTrainer(device, neuralprocess, optimizer,
#                                   num_context_range=(num_context, num_context),
#                                   num_extra_target_range=(num_target, num_target), 
#                                   print_freq=100)

# neuralprocess.training = True
# np_trainer.train(data_loader, 500)

In [10]:
# from utils import context_target_split
# from botorch.sampling.normal import SobolQMCNormalSampler
# from botorch.acquisition.monte_carlo import MCAcquisitionFunction, qExpectedImprovement
# from botorch.acquisition.monte_carlo import SampleReducingMCAcquisitionFunction
# from botorch.sampling.stochastic_samplers import StochasticSampler
# from botorch.acquisition.objective import IdentityMCObjective

# # Extract a batch from data_loader
# for batch in data_loader:
#     break

# # Use batch to create random set of context points
# x, y = batch
# x_context, y_context, _, _ = context_target_split(x[0:1], y[0:1], 
#                                                   num_context, 
#                                                   num_target)

# # Create a set of target points corresponding to entire [-pi, pi] range
# x_target = torch.Tensor(np.linspace(-5, 10, 100)).unsqueeze(1).unsqueeze(0)
# for i in range(dim-1):
#     a = torch.Tensor(np.linspace(-5, 10, 100)).unsqueeze(1).unsqueeze(0)
#     x_target = torch.cat((x_target, a), axis=2)

# best_f = Y_np.max()
# neuralprocess.training = False
# # sampler = SobolQMCNormalSampler(torch.Size([4]), seed=1234)
# # sampler = StochasticSampler(torch.Size([4]), seed=1234)
# # obj = IdentityMCObjective()
# # acq = qExpectedImprovement(neuralprocess, best_f, sampler, obj)
# posterior = neuralprocess.posterior(x_context, y_context, x_target)
# # samples = sampler(posterior)
# # print(samples.size())
# sampler = StochasticSampler(torch.Size([4]), seed=1234)
# identity_objective = IdentityMCObjective()
# samples = sampler(posterior)
# objective = identity_objective(samples)

In [11]:
## bo
from torch.utils.data import DataLoader
from neural_process import NeuralProcess
from training import NeuralProcessTrainer
from botorch.sampling.normal import SobolQMCNormalSampler
from utils import context_target_split
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.sampling.stochastic_samplers import StochasticSampler
from botorch.acquisition.objective import IdentityMCObjective

# NUM_RESTARTS = 10 if not SMOKE_TEST else 2
# RAW_SAMPLES = 500 if not SMOKE_TEST else 4
# N_CANDIDATES = min(5000, max(2000, 200 * dim)) if not SMOKE_TEST else 4
NUM_RESTARTS = 2
RAW_SAMPLES = 7
N_CANDIDATES = 4

## NP model
x_dim = 20
y_dim = 1
r_dim = 50  # Dimension of representation of context points
z_dim = 50  # Dimension of sampled latent variable
h_dim = 50  # Dimension of hidden layers in encoder and decoder
neuralprocess = NeuralProcess(x_dim, y_dim, r_dim, z_dim, h_dim)
optimizer = torch.optim.Adam(neuralprocess.parameters(), lr=3e-4)

batch_size = 8
num_context = 4
num_target = 4
np_trainer = NeuralProcessTrainer(device, neuralprocess, optimizer,
                                  num_context_range=(num_context, num_context),
                                  num_extra_target_range=(num_target, num_target), 
                                  print_freq=100)

torch.manual_seed(0)

X_np = get_initial_points(dim, n_init)
Y_np = torch.tensor(
    [eval_objective(x) for x in X_np], dtype=dtype, device=device
).unsqueeze(-1)

# sampler = SobolQMCNormalSampler(torch.Size([batch_size, RAW_SAMPLES]), seed=1234)
sampler = StochasticSampler(torch.Size([4]), seed=1234)
obj = IdentityMCObjective()

while len(Y_np) < 400:
    best_f = Y_np.max() 
    train_Y = (Y_np - Y_np.mean()) / Y_np.std()
    ackley_dataset = My_Ackley(train_x = X_np, train_y = train_Y)
    data_loader = DataLoader(ackley_dataset, batch_size=batch_size, shuffle=True)

    neuralprocess.training = True
    np_trainer.train(data_loader, 200)

    # Create a batch
    # ei = qExpectedImprovement(neuralprocess, train_Y.max())
    neuralprocess.training = False
    for batch in data_loader:
        break
    x, y = batch
    x_context, y_context, _, _ = context_target_split(x[0:1], y[0:1], 
                                                    num_context, 
                                                    num_target)
    neuralprocess.set_context_for_posterior(x_context, y_context)
    # print(neuralprocess.z_sample.size())
    # z = neuralprocess.z_sample
    # z = z.unsqueeze(1).repeat(1, RAW_SAMPLES, 1)
    # print(z.size())
    # z_flat = z.view(batch_size * RAW_SAMPLES, 50)
    # print(z_flat.size())
    # break
    ei = qExpectedImprovement(neuralprocess, best_f, sampler, obj)
    candidate, acq_value = optimize_acqf(
        ei,
        bounds=torch.stack(
            [
                torch.zeros(dim, dtype=dtype, device=device),
                torch.ones(dim, dtype=dtype, device=device),
            ]
        ),
        q=batch_size,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,
    )

    ## MC-based
    # neuralprocess.training = False
    # for batch in data_loader:
    #     break

    # # Use batch to create random set of context points
    # x, y = batch
    # x_context, y_context, _, _ = context_target_split(x[0:1], y[0:1], 
    #                                                 num_context, 
    #                                                 num_target)

    # # Create a set of target points corresponding to entire [-pi, pi] range
    # x_target = torch.Tensor(np.linspace(-5, 10, 100)).unsqueeze(1).unsqueeze(0)
    # for i in range(dim-1):
    #     a = torch.Tensor(np.linspace(-5, 10, 100)).unsqueeze(1).unsqueeze(0)
    #     x_target = torch.cat((x_target, a), axis=2)

    # posterior = neuralprocess.posterior(x_context, y_context, x_target)
    # samples = sampler(posterior)

    Y_next = torch.tensor(
        [eval_objective(x) for x in candidate], dtype=dtype, device=device
    ).unsqueeze(-1)

    # Append data
    X_np = torch.cat((X_np, candidate), axis=0)
    Y_np = torch.cat((Y_np, Y_next), axis=0)
    # print(X_ei.size(), Y_ei.size())

    # Print current status
    print(f"{len(X_np)}) Best value: {Y_np.max().item():.2e}")

iteration 100, loss 9.718
iteration 200, loss 9.692


RuntimeError: shape '[56, 50]' is invalid for input of size 400

In [ ]:
z = torch.randn(2, 50)
print(z.size())
num_points = 10
z = z.unsqueeze(1)
print(z.size())
z = z.repeat(1, num_points, 1)
print(z.size())
z_flat = z.view(2 * num_points, 50)
print(z_flat.size())

torch.Size([2, 50])
torch.Size([2, 1, 50])
torch.Size([2, 10, 50])
torch.Size([20, 50])
